## 2D Toric Hamiltoniancode

In [1]:
from tqdm import tqdm
from scipy import sparse
import numpy as np
from scipy.sparse.linalg import expm_multiply
from scipy.stats import truncnorm

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rc('text', usetex=True)
plt.rc('text.latex', preamble=r'\usepackage{amsfonts, amssymb, amsmath, physics}')
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = ['serif']
mpl.rcParams['font.serif'] = ['Times New Roman']
mpl.rcParams['font.size'] = 12


σx = np.array([[0, 1], [1, 0]])
σy = np.array([[0, -1j], [1j, 0]])
σz = np.array([[1, 0], [0, -1]])

### Hamiltonian generation

In [2]:
## Generate toric code Hamiltonian as CSR matrix

from __future__ import annotations
from typing import List, Tuple

# -----------------------------------------------------------------------------
# Layout consistent with the user's add_terms loop
# -----------------------------------------------------------------------------


def _row_lengths(m: int, n: int, pbc: bool) -> List[int]:
    """Row lengths for rows i=0..2m-1.
    Even rows length n; odd rows length n if pbc else n+1.
    """
    L = []
    for i in range(2 * m):
        if i % 2 == 0:
            L.append(n)
        else:
            L.append(n if pbc else (n + 1))
    return L


def _row_offsets(L: List[int]) -> np.ndarray:
    offs = np.zeros(len(L) + 1, dtype=np.int64)
    for i, li in enumerate(L):
        offs[i + 1] = offs[i] + li
    return offs


def _edge_index(i: int, j: int, row_offsets: np.ndarray, row_lengths: List[int]) -> int:
    if not (0 <= i < len(row_lengths)):
        raise IndexError(f"row i={i} out of range [0,{len(row_lengths)-1}]")
    if not (0 <= j < row_lengths[i]):
        raise IndexError(
            f"col j={j} out of range for row {i} (len={row_lengths[i]})")
    return int(row_offsets[i] + j)

# -----------------------------------------------------------------------------
# Check generators (match user's functions)
# -----------------------------------------------------------------------------


def _x_checks(m: int, n: int, pbc: bool) -> List[List[Tuple[int, int]]]:
    xcks: List[List[Tuple[int, int]]] = []
    for i in range(m):
        for j in range(n):
            xck = [
                ((i * 2 - 1) % (2 * m), j % n),
                ((i * 2) % (2 * m), j % n),
                ((i * 2 + 1) % (2 * m), j % n),
            ]
            if (pbc or j > 0):
                xck.append(((i * 2) % (2 * m), (j - 1) % n))
            xcks.append(sorted(xck))
        if (not pbc):
            # Add the 3-body right-boundary star per original idea (wasn't appended).
            # Coordinates use j=n for odd/even rows as in the user's comment.
            xck = [
                ((i * 2 - 1) % (2 * m), n),
                ((i * 2) % (2 * m), n - 1),
                ((i * 2 + 1) % (2 * m), n),
            ]
            xcks.append(sorted(xck))
    return xcks


def _z_checks(m: int, n: int, pbc: bool) -> List[List[Tuple[int, int]]]:
    zcks: List[List[Tuple[int, int]]] = []
    for i in range(m):
        for j in range(n):
            zck = [
                ((i * 2) % (2 * m), j % n),
                ((i * 2 + 1) % (2 * m), j % n),
                ((i * 2 + 2) % (2 * m), j % n)
            ]
            if (pbc == False) and (j == n - 1):
                zck.append(((i * 2 + 1) % (2 * m), n))
            else:
                zck.append(((i * 2 + 1) % (2 * m), (j + 1) % n))
            zcks.append(sorted(zck))
    return zcks

# -----------------------------------------------------------------------------
# Mask helpers
# -----------------------------------------------------------------------------


def _term_to_mask(term: List[Tuple[int, int]], row_offsets: np.ndarray, row_lengths: List[int]) -> int:
    mask = 0
    for (i, j) in term:
        # skip coordinates that are outside the actual row (e.g., pbc=False & j=n on even rows)
        if not (0 <= i < len(row_lengths)):
            continue
        if not (0 <= j < row_lengths[i]):
            continue
        q = _edge_index(i, j, row_offsets, row_lengths)
        mask |= (1 << q)
    return mask

# -----------------------------------------------------------------------------
# Main: fast builder for H as CSR
# -----------------------------------------------------------------------------


def generate_toric_code(m: int, n: int, pbc: bool = False, *, J_e: float = 1.0, J_m: float = 1.0,
                        include_right_boundary_xstars: bool = False) -> sparse.csr_matrix:
    """Build the toric-code Hamiltonian as a CSR matrix, matching user's layout.

    Parameters
    ----------
    m, n : lattice parameters from user's code
    pbc  : if False, odd rows have n+1 sites (extra right-boundary qubits)
    J_e, J_m : coupling strengths for X-stars and Z-plaquettes
    include_right_boundary_xstars : if True, add the 3-body right-rim stars that
        the user's original function constructed but did not append.

    Returns
    -------
    H : scipy.sparse.csr_matrix of shape (D, D) with D = 2^(2*m*n + (m if not pbc else 0))
    """
    # Row structure & Hilbert space dimension
    row_lengths = _row_lengths(m, n, pbc)
    row_offsets = _row_offsets(row_lengths)
    N = int(row_offsets[-1])  # total number of qubits
    D = 1 << N

    # Build masks for all checks
    x_terms = _x_checks(m, n, pbc)
    z_terms = _z_checks(m, n, pbc)
    x_masks = [_term_to_mask(t, row_offsets, row_lengths) for t in x_terms]
    z_masks = [_term_to_mask(t, row_offsets, row_lengths) for t in z_terms]

    # Pre-allocate lists for COO assembly
    rows_parts: List[np.ndarray] = []
    cols_parts: List[np.ndarray] = []
    data_parts: List[np.ndarray] = []

    # X part: for each mask, add entries at (k^mask, k) with weight -J_e
    idx = np.arange(D, dtype=np.uint64)
    base_cols = idx.astype(np.int64, copy=False)
    for mask in x_masks:
        if mask == 0:
            # degenerate term; skip
            continue
        dest = np.bitwise_xor(idx, np.uint64(
            mask)).astype(np.int64, copy=False)
        rows_parts.append(dest)
        cols_parts.append(base_cols)
        data_parts.append(np.full(D, -J_e, dtype=np.complex128))

    # Z part: cumulative diagonal from all plaquettes
    def _mask_phase(mask: int) -> np.ndarray:
        """Return ±1 phase vector for the diagonal action of one Z-term.
        dtype-safe parity using int8 accumulator.
        """
        parity = np.zeros(D, dtype=np.int8)
        mm = mask
        while mm:
            q = (mm & -mm).bit_length() - 1  # lowest set bit
            # Cast to int8 before XOR to avoid ufunc dtype error
            parity ^= ((idx >> q) & np.uint64(1)).astype(np.int8)
            mm &= mm - 1
        return (1 - 2 * parity).astype(np.int8)

    if z_masks:
        diag_sum = np.zeros(D, dtype=np.int16)
        for mask in z_masks:
            if mask == 0:
                continue
            diag_sum += _mask_phase(mask)
        rows_parts.append(np.arange(D, dtype=np.int64))
        cols_parts.append(np.arange(D, dtype=np.int64))
        data_parts.append((-J_m * diag_sum).astype(np.complex128))

    # Assemble COO -> CSR
    if rows_parts:
        rows = np.concatenate(rows_parts)
        cols = np.concatenate(cols_parts)
        data = np.concatenate(data_parts)
        H = sparse.coo_matrix((data, (rows, cols)), shape=(D, D)).tocsr()
    else:
        H = sparse.csr_matrix((D, D), dtype=np.complex128)

    return H

In [3]:
# Test run
m = 2
n = 4
pbc = False

H = generate_toric_code(m, n, pbc=pbc)
H = H / sparse.linalg.norm(H, 2)
eigenvalues, eigenvectors = sparse.linalg.eigsh(H, k = 10, which = "SA")
print(np.sort(eigenvalues))
sparse.save_npz('data/Hamiltonian_%d_%d_%d.npz' % (m,n, pbc), H)

[-1.         -1.         -0.88888889 -0.88888889 -0.88888889 -0.88888889
 -0.88888889 -0.88888889 -0.88888889 -0.88888889]


## Initial states generation
* Each state is first sampled from a Haar random distribution
* Multiply by $\exp(-\beta H)$, and then normalize

In [4]:
def trial(β = 10, N_state = 10, N_sample = 100, T = 30, pbc=True):
    # Function to generate data
    N = 2 * m * n
    if (pbc == False):
        N = N + m
    D = 2 ** N

    # draw times
    t = truncnorm(-T, T, loc=0.0, scale=T).rvs(size=N_sample)
    t = np.sort(t)
    pos_zero = np.argmax(t >= 0)
    neg_t = t[:pos_zero]
    pos_t = t[pos_zero:]
    neg_t = neg_t[::-1]
    # print(neg_t, pos_t)
    t = np.concatenate((pos_t, neg_t))

    def generate_haar_random_state(d, N_state):
        state = np.random.randn(d, N_state) + 1j * np.random.randn(d, N_state)
        return state / np.linalg.norm(state, axis = 0)
    
    
    s = generate_haar_random_state(D, N_state)    
    s = expm_multiply(-β * H, s)
    s = s / np.linalg.norm(s, axis = 0)
    
    # precompute list of M^(k)
    M_list = []

    st = np.copy(s)
    for k in tqdm(range(len(pos_t))):
        M_k = np.zeros((N_state, N_state), dtype=complex)
        dt = pos_t[k] - (pos_t[k-1] if k > 0 else 0)
        st = expm_multiply(1j * dt * H, st)
        M_k = np.einsum('ij,ik->jk', s.conj(), st)
        # print(s.shape, R.shape, M_k.shape)
        M_list.append(M_k)
    
    st = np.copy(s)
    for k in tqdm(range(len(neg_t))):
        M_k = np.zeros((N_state, N_state), dtype=complex)
        dt = neg_t[k] - (neg_t[k-1] if k > 0 else 0)
        st = expm_multiply(1j * dt * H, st)
        M_k = np.einsum('ij,ik->jk', s.conj(), st)
        # print(s.shape, R.shape, M_k.shape)
        M_list.append(M_k)

    # Rounding to +-1+-i
    for k in range(N_sample):
        for i in range(N_state):
            for j in range(N_state):
                p_re = (M_list[k][i, j].real + 1) / 2
                p_im = (M_list[k][i, j].imag + 1) / 2
                r = np.random.uniform(0, 1)
                if r < p_re:
                    x = 1
                else:
                    x = -1
                r = np.random.uniform(0, 1)
                if r < p_im:
                    x += 1j
                else:
                    x -= 1j
                M_list[k][i, j] = x
    mu = -1.0
    W = sum(M_list[k] * np.exp(-1j * mu * t[k]) for k in range(N_sample))
    W /= N_sample
    sigs = np.linalg.svdvals(W)

    num_gs = 4 if pbc else 2
    Phi = np.zeros((N_state, num_gs), dtype=complex)

    Phi = np.einsum('ij,ik->jk', s, eigenvectors[:, :num_gs])
    Phi_n = np.zeros((N_state, num_gs), dtype=complex)
    for j in range(num_gs):
        Phi_n[:, j] = Phi[:, j] / np.linalg.norm(Phi[:, j])
    lambda_min = np.min(np.linalg.eigvalsh(Phi_n.conj().T @ Phi_n))        
    print(lambda_min)
    return sigs

### Main algorithm

In [ ]:
paras = [[10, 15, 10, 300], [15, 15, 10, 300], [10, 25, 10, 300]]
readFromFile = True # Whether or not to read data from file

for pbc in [True, False]:
    H = sparse.load_npz('data/Hamiltonian_%d_%d_%d.npz' % (m,n,pbc))
    eigenvalues, eigenvectors = sparse.linalg.eigsh(H, k = 8, which = "SA")
    so = eigenvalues.argsort()
    eigenvalues = eigenvalues[so]
    eigenvectors = eigenvectors[:, so]
    print(eigenvalues)
    for β, N_state, T, N_sample in paras:
        print('----------------------------------')
        print(f'Parameters: beta={β}, N_state={N_state}, T={T}, N_sample={N_sample}, pbc={pbc}')

        if (not readFromFile):
            num = 10
            sig_list = []
            for i in range(num):
                print(f'Trial #{i}:')
                sig_list.append(trial(β, N_state, N_sample, T, pbc))

            sig_list = np.array(sig_list)
            N_state = sig_list.shape[1]
            sv = np.zeros(N_state)
            for i in range(N_state):
                sv[i] = np.mean(sig_list[:, i])
            sv_err = np.zeros(N_state)
            for i in range(N_state):
                sv_err[i] = -1e10
                for j in range(num):
                    sv_err[i] = max(sv_err[i], abs(sig_list[j, i] - sv[i]))

            np.savetxt('data/sv_m%d_n%d_pbc%d_beta%d_T%d_L%d' % (m, n, pbc, β, T, N_state), sv)
            np.savetxt('data/sv_err_m%d_n%d_pbc%d_beta%d_T%d_L%d' % (m, n, pbc, β, T, N_state), sv_err)
        else:
            sv = np.loadtxt('data/sv_m%d_n%d_pbc%d_beta%d_T%d_L%d' % (m, n, pbc, β, T, N_state))
            sv_err = np.loadtxt('data/sv_err_m%d_n%d_pbc%d_beta%d_T%d_L%d' % (m, n, pbc, β, T, N_state))
        plt.figure(figsize=(4, 3))
        x = np.arange(N_state)
        plt.axhline(1 * N_state / 15, color='red', linestyle='--', linewidth=2, label=r'$\tau = L/15$')
        plt.bar(x+1, sv, label=r'$\sigma(G(-1))$', color='blue', width=0.4)

        plt.errorbar(x + 1, sv, yerr=sv_err, fmt='o', ms=2.5, color="darkorange")

        plt.ylim([0., np.max(sv + sv_err) + 0.2])

        if N_state < 20:
            xp = np.arange(N_state)
        else:
            xp = np.arange(N_state // 2)  * 2
        plt.xticks(xp+1, fontsize=12)
        plt.yticks(fontsize=12)
        plt.legend(fontsize=12)
        plt.savefig('figs/Toric_svd_%d_%d_%d_beta_%d_state_%d_sample_%d_T%d.pdf' % (m, n, pbc, β, N_state, N_sample, T), bbox_inches='tight')
        plt.close()


[-1.   -1.   -1.   -1.   -0.75 -0.75 -0.75 -0.75]
----------------------------------
Parameters: beta=10, N_state=15, T=10, N_sample=300, pbc=True
----------------------------------
Parameters: beta=15, N_state=15, T=10, N_sample=300, pbc=True
----------------------------------
Parameters: beta=10, N_state=25, T=10, N_sample=300, pbc=True
[-1.         -1.         -0.88888889 -0.88888889 -0.88888889 -0.88888889
 -0.88888889 -0.88888889]
----------------------------------
Parameters: beta=10, N_state=15, T=10, N_sample=300, pbc=False
----------------------------------
Parameters: beta=15, N_state=15, T=10, N_sample=300, pbc=False
----------------------------------
Parameters: beta=10, N_state=25, T=10, N_sample=300, pbc=False
